题目$\frac{1}{x}+\frac{1}{y}<\frac{1}{n}$ ,根据题意可知n<x<=y

参考地址：https://blog.csdn.net/piaocoder/article/details/47954385

# 1. 用因子对表示解

先回忆已知等价式：

$$
\frac{1}{x} + \frac{1}{y} = \frac{1}{n}
\quad\Longleftrightarrow\quad
(x-n)(y-n)=n^2.
$$

令：

$$
a = x-n,\qquad b = y-n.
$$

注意 $x,y,n$ 都取正整数并且我们要求 $x>n,\; y>n$，所以 $a,b$ 都是正整数。  
于是每一对正整数因子 $(a,b)$ 满足 $ab=n^2$ 就对应一组解：

$$
(x,y)=(n+a,n+b).
$$

---

# 2. 有序对与无序对的关系

- $n^2$ 的正因子总数记作 $\tau(n^2)$.  
- $\tau(n^2)$ 精确等于满足 $ab=n^2$ 的 **有序** 正因子对 $(a,b)$ 的数量（因为每个因子 $d$ 对应 $(d, n^2/d)$）。  
- $(x,y)$ 视为 **不计顺序** 或者要求 $x \le y$. 把有序的因子对变为不计顺序的对，需要把互换顺序视为同一解。

---

# 3. 平方数的特殊性（中轴对）

一般地，把有序对数量转为不计顺序的数量，若没有自反对会是：

$$
\frac{\tau(n^2)}{2}.
$$

但当 $n^2$ 是完全平方时，存在自反对 $(\sqrt{n^2},\sqrt{n^2})=(n,n)$（即 $a=b$），它在有序对里只出现一次，在无序对里也只算一次。

因此总的不计顺序的因子对数（即满足 $a \le b$ 的对数）为：

$$
\#\{(a,b):\ ab=n^2,\; a \le b\}=\frac{\tau(n^2)+1}{2}.
$$

（直观：把所有 $\tau(n^2)$ 个有序对配成 $\tau(n^2)/2$ 对互换对，再加上中轴自反的那一个。）

---

# 4. 把“解的数量 = 4,000,000”转换成 $\tau(n^2)$ 的等式

题意给出：满足 $x \le y$ 的正整数解个数为 $4{,}000{,}000$. 根据上面等式：

$$
\frac{\tau(n^2)+1}{2}=4{,}000{,}000.
$$

解这个等式得：

$$
\tau(n^2)+1 = 8{,}000{,}000 \quad\Longrightarrow\quad \tau(n^2)=7{,}999{,}999.
$$

分解：  

$$
7,999,999 = 7 \times 199 \times 5743,
$$  

且这三个因子都是素数。 

---

## 5) 使 $n$ 最小的构造与排列原则  

给定 $(2e_i+1)$ 的乘积固定，要使  

$$
n=\prod_{i=1}^k p_i^{e_i}
$$  

最小，应将 **最大的指数配给最小的质数**。  
（重排不等式/贪心原则：大指数配小质数能最小化积。）  

把三个因子直接作为三个 $(2e_i+1)$：  

$$
(2e_1+1,\;2e_2+1,\;2e_3+1)=(5743,\;199,\;7),
$$  

对应：  

$$
(e_1,e_2,e_3)=\left(\frac{5743-1}{2},\;\frac{199-1}{2},\;\frac{7-1}{2}\right)=(2871,\;99,\;3).
$$  

将它们依大到小配给最小的质数 $2,3,5$：  

$$
n = 2^{2871} \cdot 3^{99} \cdot 5^3.
$$  

---

## 6) 校验  

$$
\tau(n^2) = (2\cdot 2871+1)(2\cdot 99+1)(2\cdot 3+1)
= 5743 \cdot 199 \cdot 7 = 7,999,999,
$$  

$$
\frac{\tau(n^2)+1}{2}=4,000,000.
$$  

---

# 7. （补充）为什么 $\tau(n^2)$ 总是奇数？

若

$$
n=\prod_i p_i^{e_i},
$$

则

$$
n^2=\prod_i p_i^{2e_i},\qquad \tau(n^2)=\prod_i (2e_i+1).
$$

每个因子 $2e_i+1$ 都是奇数，奇数的乘积仍为奇数。  
所以 $\tau(n^2)$ 必是奇数；因此 $\tau(n^2)=7{,}999{,}999$ 是合理的奇数目标值。




In [1]:
import math
from functools import lru_cache

# 取前若干质数（通常前15-20个质数就足够）
PRIMES = [2,3,5,7,11,13,17,19,23,29,31,37,41,43,47,53,59,61]

def find_min_n_for_m(m, primes=PRIMES):
    """
    返回最小的 n 使得 (tau(n^2)+1)/2 >= m。
    返回 (n, factorization) 其中 factorization 为 [(p,e),...]
    """
    target = 2*m - 1  # 需要 tau(n^2) >= target
    best = {"n": None, "factors": None}

    # 初始上界：可以设置为非常大整数
    best["n"] = 10**200

    # 向上估计允许的最大指数（为了开始，可以用较大值）
    # 这里设置为 60，足够大；可根据 target 调整
    MAX_EXP_INIT = 60

    def dfs(idx, prev_exp, curr_tau, curr_n, curr_factors):
        """
        idx: 当前使用 primes[idx]
        prev_exp: 下一质数的指数上限（保证指数非增）
        curr_tau: 当前乘积 tau_part = prod(2e_i+1)
        curr_n: 当前构造的 n
        curr_factors: 当前质因子列表
        """
        # 剪枝：如果当前 n 已经 >= 最优，直接返回
        if curr_n >= best["n"]:
            return

        # 如果已经满足目标，更新最优
        if curr_tau >= target:
            if curr_n < best["n"]:
                best["n"] = curr_n
                best["factors"] = curr_factors.copy()
            return

        # 如果没有更多质数可用，返回
        if idx >= len(primes):
            return

        p = primes[idx]

        # 为了让较大指数先被尝试（把大指数放到小质数），我们从 prev_exp 到 1 递减试
        # prev_exp 的初值可设得很大（例如 60）
        for e in range(prev_exp, 0, -1):
            # 防止 curr_n * p**e 超过当前最优值（快速剪枝）
            # 使用 pow with third arg None to compute quickly (no mod)
            try:
                new_n = curr_n * pow(p, e)
            except OverflowError:
                break
            if new_n >= best["n"]:
                continue

            new_tau = curr_tau * (2*e + 1)
            # 剪枝：如果新tau * (最小可能以后因子贡献) 仍然不能达到 target, we still continue
            # but computing a cheap lower bound for remaining primes is complicated;
            # we rely on direct DFS order and n-based pruning.

            curr_factors.append((p, e))
            dfs(idx + 1, e, new_tau, new_n, curr_factors)
            curr_factors.pop()

    # 启动DFS
    dfs(0, MAX_EXP_INIT, 1, 1, [])
    return best["n"], best["factors"]

In [2]:
m=1000
n, fac = find_min_n_for_m(m)
print(f"m={m}, min n = {n}")
print("factors:", fac)

m=1000, min n = 180180
factors: [(2, 2), (3, 2), (5, 1), (7, 1), (11, 1), (13, 1)]


In [3]:
m=4000000
n, fac = find_min_n_for_m(m)
print(f"m={m}, min n = {n}")
print("factors:", fac)

m=4000000, min n = 9350130049860600
factors: [(2, 3), (3, 3), (5, 2), (7, 2), (11, 1), (13, 1), (17, 1), (19, 1), (23, 1), (29, 1), (31, 1), (37, 1)]
